In [1]:
import pandas as pd
from nervaluate import Evaluator
from nltk.tokenize import word_tokenize
from nervaluate import collect_named_entities
from nervaluate import compute_metrics

In [2]:
prediction_file = "/home/lhoang2/Notebooks/RCT_Methodology_Extraction/DATA_07232022/RESULTS_07232022/prediction_TEST_SET_lr55_epoch20_random_similarity_sampled_BERT_07232022.csv"
prediction_df = pd.read_csv(prediction_file)

In [3]:
prediction_df = prediction_df[["sentence_id","sentence","word_labels","pred_labels"]]
display(prediction_df.head(4))

,sentence_id,sentence,word_labels,pred_labels
0,S2223,Safety outcomes were assessed throughout the 3...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,S2268,"Based on non-compartmental analysis , the mean...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,S2278,4 ) . 4 ) . 4 ) .,"O,O,O,O,O,O,O,O,O","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
3,S2215,Asthma exacerbations were defined as new or in...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [4]:
all_test_data_withPMCIDs = "/home/lhoang2/Notebooks/RCT_Methodology_Extraction/DATA_07232022/RESULTS_07232022/ALL_TESTINGDATA_07232022.csv"
all_test_data_withPMCIDs_df = pd.read_csv(all_test_data_withPMCIDs)
display(all_test_data_withPMCIDs_df.head(2))

,Unnamed: 0,sentence_id,sentence,word_labels,PMCID
0,0,S0,TITLE Double-Blind Randomized Trials of Single...,"O,B-Blinding_Double_Blind,O,O,O,O,O,O,O,O,O,O,...",PMC3321505
1,18,S1,ABSTRACT,O,PMC3321505


In [5]:
merged_df = pd.merge(prediction_df,all_test_data_withPMCIDs_df,how="inner",on=["sentence_id","sentence","word_labels"])
display(merged_df.head(2))
print (len(merged_df))

,sentence_id,sentence,word_labels,pred_labels,Unnamed: 0,PMCID
0,S149,"Results In total , 149 patients were randomise...","O,O,O,O,B-Sample_Size_Actual_at_Enrollment,O,O...","['O', 'O', 'O', 'O', 'B-Sample_Size_Actual_at_...",4618,PMC4066691
1,S1316,The corresponding author had full access to al...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",40571,PMC3386495


1386


In [6]:
PMCIDs = set(merged_df["PMCID"])
print (PMCIDs)

trial_design_type_total_TP = 0
trial_design_type_total_FP = 0
trial_design_type_total_FN = 0

randomization_type_total_TP = 0
randomization_type_total_FP = 0
randomization_type_total_FN = 0

blinding_type_total_TP = 0
blinding_type_total_FP = 0
blinding_type_total_FN = 0

phase_total_TP = 0
phase_total_FP = 0
phase_total_FN = 0

settings_total_TP = 0
settings_total_FN = 0
settings_total_FP = 0

total_TP = 0
total_FP = 0
total_FN = 0


for PMCID in PMCIDs:
    if PMCID != "":
        print (PMCID)
        PMCID_df = merged_df[merged_df["PMCID"] == PMCID]
        PMCID_true_labels = PMCID_df["word_labels"].to_list()
        PMCID_pred_labels = PMCID_df["pred_labels"].to_list()
        PMCID_sentences = PMCID_df["sentence"].to_list()

        #process true labels:
        unique_true_labels = []
        for item in PMCID_true_labels:
            item = item.split(",")
            for label in item:
                if (label != "O") and (label not in unique_true_labels):
                    unique_true_labels.append(label)
        
        #process pred labels:
        unique_pred_labels = []
        for item in PMCID_pred_labels:
            if isinstance(item, str):
                item = item.replace("\n","")
                item = item.replace("]","")
                item = item.replace("[","")
                item = item.replace("'","")
                item = item.replace(" ","")
                item = item.split(",")
                for label in item:
                    if (label != "O") and (label not in unique_pred_labels):
                        unique_pred_labels.append(label)

        # Trial design type:
        trial_design_type_entity = ["Design_Parallel_Group", 
                                    "Design_Crossover", 
                                    "Design_Factorial", 
                                    "Design_N-of_1", 
                                    "Design_Other"]
        trial_design_type_TRUE = []
        for true_label in unique_true_labels:
            if (true_label.startswith("I-Design")) or (true_label.startswith("B-Design")):
                true_label = true_label.replace("B-","")
                true_label = true_label.replace("I-","")
                if true_label in trial_design_type_entity:
                    trial_design_type_TRUE.append(true_label)
        
        trial_design_type_PRED = []
        for pred_label in unique_pred_labels:
            if (pred_label.startswith("I-Design")) or (pred_label.startswith("B-Design")):
                pred_label = pred_label.replace("B-","")
                pred_label = pred_label.replace("I-","")
                if pred_label in trial_design_type_entity:
                    trial_design_type_PRED.append(pred_label)
                
        trial_design_type_TRUE= set(trial_design_type_TRUE) 
        trial_design_type_PRED= set(trial_design_type_PRED) 

        trial_design_true_positive = (len(trial_design_type_TRUE.intersection(trial_design_type_PRED)))
        trial_design_false_negative = (len(trial_design_type_TRUE.difference(trial_design_type_PRED)))
        trial_design_false_positive = (len(trial_design_type_PRED.difference(trial_design_type_TRUE)))
        
        trial_design_type_total_TP += trial_design_true_positive
        trial_design_type_total_FN += trial_design_false_negative
        trial_design_type_total_FP += trial_design_false_positive
        
        # Randomization type:
        randomization_type_entity = ["Randomization_Type_Minimization",
                                    "Randomization_Type_Stratified",
                                    "Randomization_Type_Simple","Randomization_Type_Block",
                                    "Randomization_Type_Other"]
        randomization_type_TRUE = []
        randomization_type_PRED = []
        
        for true_label in unique_true_labels:
            if (true_label.startswith("I-Randomization_Type")) or (true_label.startswith("B-Randomization_Type")):
                true_label = true_label.replace("B-","")
                true_label = true_label.replace("I-","")
                if true_label in randomization_type_entity:
                    randomization_type_TRUE.append(true_label)
        
        for pred_label in unique_pred_labels:
            if (pred_label.startswith("I-Randomization_Type")) or (pred_label.startswith("B-Randomization_Type")):
                pred_label = pred_label.replace("B-","")
                pred_label = pred_label.replace("I-","")
                if pred_label in randomization_type_entity:
                    randomization_type_PRED.append(pred_label)
                    
        randomization_type_TRUE= set(randomization_type_TRUE) 
        randomization_type_PRED= set(randomization_type_PRED) 

        randomization_true_positive = (len(randomization_type_TRUE.intersection(randomization_type_PRED)))
        randomization_false_negative = (len(randomization_type_TRUE.difference(randomization_type_PRED)))
        randomization_false_positive = (len(randomization_type_PRED.difference(randomization_type_TRUE)))
        
        randomization_type_total_TP += randomization_true_positive
        randomization_type_total_FN += randomization_false_negative
        randomization_type_total_FP += randomization_false_positive
        
        
        # Blinding method type:
        blinding_type_entity = ["Blinding_Double_Blind",
                                    "Blinding_Single_Blind",
                                    "Blinding_Open_Label","Blinding_Triple_Blind",
                                    "Blinding_Quadruple_Blind"]
        blinding_type_TRUE = []
        blinding_type_PRED = []
        
        for true_label in unique_true_labels:
            if (true_label.startswith("I-Blinding")) or (true_label.startswith("B-Blinding")):
                true_label = true_label.replace("B-","")
                true_label = true_label.replace("I-","")
                if true_label in blinding_type_entity:
                    blinding_type_TRUE.append(true_label)
        
        for pred_label in unique_pred_labels:
            if (pred_label.startswith("I-Blinding")) or (pred_label.startswith("B-Blinding")):
                pred_label = pred_label.replace("B-","")
                pred_label = pred_label.replace("I-","")
                if pred_label in blinding_type_entity:
                    blinding_type_PRED.append(pred_label)
        
        blinding_type_TRUE= set(blinding_type_TRUE) 
        blinding_type_PRED= set(blinding_type_PRED) 

        blinding_true_positive = (len(blinding_type_TRUE.intersection(blinding_type_PRED)))
        blinding_false_negative = (len(blinding_type_TRUE.difference(blinding_type_PRED)))
        blinding_false_positive = (len(blinding_type_PRED.difference(blinding_type_TRUE)))
        
        blinding_type_total_TP += blinding_true_positive
        blinding_type_total_FN += blinding_false_negative
        blinding_type_total_FP += blinding_false_positive
        
        # Phase:     
        phase_entity = ["Design_Phase_1",
                                    "Design_Phase_2",
                                    "Design_Phase_3",
                                    "Design_Phase_4"]
        phase_TRUE = []
        phase_PRED = []
        
        for true_label in unique_true_labels:
            if (true_label.startswith("I-Design_Phase")) or (true_label.startswith("B-Design_Phase")):
                true_label = true_label.replace("B-","")
                true_label = true_label.replace("I-","")
                if true_label in phase_entity:
                    phase_TRUE.append(true_label)
                    
        for pred_label in unique_pred_labels:
            if (pred_label.startswith("I-Design_Phase")) or (pred_label.startswith("B-Design_Phase")):
                pred_label = pred_label.replace("B-","")
                pred_label = pred_label.replace("I-","")
                if pred_label in phase_entity:
                    phase_PRED.append(pred_label)
        
        phase_TRUE= set(phase_TRUE) 
        phase_PRED= set(phase_PRED) 
#         print ("True phase: ", phase_TRUE)
#         print ("Pred phase:", phase_PRED)
        
#         for sentence in PMCID_sentences:
#             if "phase" in sentence or "Phase" in sentence:
#                 print (sentence)

        phase_true_positive = (len(phase_TRUE.intersection(phase_PRED)))
        phase_false_negative = (len(phase_TRUE.difference(phase_PRED)))
        phase_false_positive = (len(phase_PRED.difference(phase_TRUE)))
        
        
        phase_total_TP += phase_true_positive
        phase_total_FN += phase_false_negative
        phase_total_FP += phase_false_positive
        
        

        # Settings:
        settings_entity = ["Settings_Multicenter",
                                    "Settings_Single_Center"]
        settings_TRUE = []
        settings_PRED = []
        
        for true_label in unique_true_labels:
            if (true_label.startswith("I-Settings")) or (true_label.startswith("B-Settings")):
                true_label = true_label.replace("B-","")
                true_label = true_label.replace("I-","")
                if true_label in settings_entity:
                    settings_TRUE.append(true_label)
                    
        for pred_label in unique_pred_labels:
            if (pred_label.startswith("I-Settings")) or (pred_label.startswith("B-Settings")):
                pred_label = pred_label.replace("B-","")
                pred_label = pred_label.replace("I-","")
                if pred_label in settings_entity:
                    settings_PRED.append(pred_label)
        
        settings_TRUE= set(settings_TRUE) 
        settings_PRED= set(settings_PRED) 

        settings_true_positive = (len(settings_TRUE.intersection(settings_PRED)))
        settings_false_negative = (len(settings_TRUE.difference(settings_PRED)))
        settings_false_positive = (len(settings_PRED.difference(settings_TRUE)))
        
        settings_total_TP += settings_true_positive
        settings_total_FN += settings_false_negative
        settings_total_FP += settings_false_positive
                
        total_TP += settings_total_TP + trial_design_type_total_TP + randomization_type_total_TP + blinding_type_total_TP + phase_total_TP
        total_FP += settings_total_FP + trial_design_type_total_FP + randomization_type_total_FP + blinding_type_total_FP + phase_total_FP
        total_FN += settings_total_FN + trial_design_type_total_FN + randomization_type_total_FN + blinding_type_total_FN + phase_total_FN


print ("\n")
print("TRIAL DESIGN TYPE:")
trial_design_type_P = trial_design_type_total_TP / (trial_design_type_total_TP + trial_design_type_total_FP)
trial_design_type_R = trial_design_type_total_TP / (trial_design_type_total_TP + trial_design_type_total_FN) 
trial_design_type_F1 = 2*(trial_design_type_P*trial_design_type_R)/(trial_design_type_P+trial_design_type_R)
print (trial_design_type_P)
print (trial_design_type_R)
print (trial_design_type_F1)

print ("\n")
print("RANDOMIZATION TYPE:")
randomization_type_P = randomization_type_total_TP / (randomization_type_total_TP + randomization_type_total_FP)
randomization_type_R = randomization_type_total_TP / (randomization_type_total_TP + randomization_type_total_FN) 
randomization_type_F1 = 2*(randomization_type_P*randomization_type_R)/(randomization_type_P+randomization_type_R)
print (randomization_type_P)
print (randomization_type_R)
print (randomization_type_F1)

print ("\n")
print("BLINDING TYPE:")
blinding_type_P = blinding_type_total_TP / (blinding_type_total_TP + blinding_type_total_FP)
blinding_type_R = blinding_type_total_TP / (blinding_type_total_TP + blinding_type_total_FN) 
blinding_type_F1 = 2*(blinding_type_P*blinding_type_R)/(blinding_type_P+blinding_type_R)
print (blinding_type_P)
print (blinding_type_R)
print (blinding_type_F1)

print ("\n")
print("PHASE:")
print ("TP: ",phase_total_TP,"FP: ", phase_total_FP,"FN: ", phase_total_FN)
phase_type_P = phase_total_TP / (phase_total_TP + phase_total_FP)
phase_type_R = phase_total_TP / (phase_total_TP + phase_total_FN) 
if phase_type_P!= 0 and phase_type_R != 0:
    phase_type_F1 = 2*(phase_type_P*phase_type_R)/(phase_type_P+phase_type_R)
else:
    phase_type_F1 = 0
print (phase_type_P)
print (phase_type_R)
print (phase_type_F1)


print ("\n")
print("SETTINGS:")
print (settings_total_TP, settings_total_TP, settings_total_FP)
settings_type_P = settings_total_TP / (settings_total_TP + settings_total_FP)
settings_type_R = settings_total_TP / (settings_total_TP + settings_total_FN) 
settings_type_F1 = 2*(settings_type_P*settings_type_R)/(settings_type_P+settings_type_R)
print (settings_type_P)
print (settings_type_R)
print (settings_type_F1)


print ("\n")
print("Everything:")
P = total_TP / (total_TP + total_FP)
R = total_TP / (total_TP + total_FN) 
F1 = 2*(P*R)/(P+R)
print (P)
print (R)
print (F1)

{'PMC3076731', 'PMC3668094', 'PMC4145444', 'PMC3109952', 'PMC4066691', 'PMC3242163', 'PMC3002766', 'PMC3623038', 'PMC3321505', 'PMC4469977', 'PMC3386495', 'PMC3756454', 'PMC4797126', 'PMC3103669', 'PMC3641608'}
PMC3076731
0 0 0
PMC3668094
0 0 0
PMC4145444
0 0 0
PMC3109952
0 0 0
PMC4066691
1 0 0
PMC3242163
0 0 0
PMC3002766
0 0 0
PMC3623038
0 0 0
PMC3321505
0 0 0
PMC4469977
0 0 0
PMC3386495
0 0 1
PMC3756454
0 0 0
PMC4797126
0 0 0
PMC3103669
0 0 0
PMC3641608
0 0 0


TRIAL DESIGN TYPE:
1.0
1.0
1.0


RANDOMIZATION TYPE:
0.9166666666666666
0.6111111111111112
0.7333333333333334


BLINDING TYPE:
0.9230769230769231
0.9230769230769231
0.9230769230769231


PHASE:
TP:  1 FP:  1 FN:  0
0.5
1.0
0.6666666666666666


SETTINGS:
11 11 1
0.9166666666666666
0.8461538461538461
0.8799999999999999


Everything:
0.9397321428571429
0.8287401574803149
0.8807531380753139
